In [1]:
!pip install datasets
!pip install transformers
!pip install transformers accelerate evaluate datasets peft -q
!pip install --no-cache-dir transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 17.4 MB/s eta 0:00:00


In [2]:
from __future__ import print_function

import torch
import torch.nn as nn
from transformers import DebertaV2PreTrainedModel, DebertaV2Model
from typing import Optional, Tuple, Union

from transformers.modeling_outputs import MultipleChoiceModelOutput
from transformers.activations import ACT2FN

class ContextPooler(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.pooler_hidden_size, config.pooler_hidden_size)
        self.dropout = StableDropout(config.pooler_dropout)
        self.config = config

    def forward(self, hidden_states):
        # We "pool" the model by simply taking the hidden state corresponding
        # to the first token.

        context_token = hidden_states[:, 0]
        context_token = self.dropout(context_token)
        pooled_output = self.dense(context_token)
        pooled_output = ACT2FN[self.config.pooler_hidden_act](pooled_output)
        return pooled_output

    @property
    def output_dim(self):
        return self.config.hidden_size

class StableDropout(nn.Module):
    """
    Optimized dropout module for stabilizing the training

    Args:
        drop_prob (float): the dropout probabilities
    """

    def __init__(self, drop_prob):
        super().__init__()
        self.drop_prob = drop_prob
        self.count = 0
        self.context_stack = None

    def forward(self, x):
        """
        Call the module

        Args:
            x (`torch.tensor`): The input tensor to apply dropout
        """
        if self.training and self.drop_prob > 0:
            return XDropout.apply(x, self.get_context())
        return x

    def clear_context(self):
        self.count = 0
        self.context_stack = None

    def init_context(self, reuse_mask=True, scale=1):
        if self.context_stack is None:
            self.context_stack = []
        self.count = 0
        for c in self.context_stack:
            c.reuse_mask = reuse_mask
            c.scale = scale

    def get_context(self):
        if self.context_stack is not None:
            if self.count >= len(self.context_stack):
                self.context_stack.append(DropoutContext())
            ctx = self.context_stack[self.count]
            ctx.dropout = self.drop_prob
            self.count += 1
            return ctx
        else:
            return self.drop_prob

class DropoutContext(object):
    def __init__(self):
        self.dropout = 0
        self.mask = None
        self.scale = 1
        self.reuse_mask = True


# Copied from transformers.models.deberta.modeling_deberta.get_mask
def get_mask(input, local_context):
    if not isinstance(local_context, DropoutContext):
        dropout = local_context
        mask = None
    else:
        dropout = local_context.dropout
        dropout *= local_context.scale
        mask = local_context.mask if local_context.reuse_mask else None

    if dropout > 0 and mask is None:
        mask = (1 - torch.empty_like(input).bernoulli_(1 - dropout)).to(torch.bool)

    if isinstance(local_context, DropoutContext):
        if local_context.mask is None:
            local_context.mask = mask

    return mask, dropout
class XDropout(torch.autograd.Function):
    """Optimized dropout function to save computation and memory by using mask operation instead of multiplication."""

    @staticmethod
    def forward(ctx, input, local_ctx):
        mask, dropout = get_mask(input, local_ctx)
        ctx.scale = 1.0 / (1 - dropout)
        if dropout > 0:
            ctx.save_for_backward(mask)
            return input.masked_fill(mask, 0) * ctx.scale
        else:
            return input

    @staticmethod
    def backward(ctx, grad_output):
        if ctx.scale > 1:
            (mask,) = ctx.saved_tensors
            return grad_output.masked_fill(mask, 0) * ctx.scale, None
        else:
            return grad_output, None

    @staticmethod
    def symbolic(g: torch._C.Graph, input: torch._C.Value, local_ctx: Union[float, DropoutContext]) -> torch._C.Value:
        from torch.onnx import symbolic_opset12

        dropout_p = local_ctx
        if isinstance(local_ctx, DropoutContext):
            dropout_p = local_ctx.dropout
        # StableDropout only calls this function when training.
        train = True
        # TODO: We should check if the opset_version being used to export
        # is > 12 here, but there's no good way to do that. As-is, if the
        # opset_version < 12, export will fail with a CheckerError.
        # Once https://github.com/pytorch/pytorch/issues/78391 is fixed, do something like:
        # if opset_version < 12:
        #   return torch.onnx.symbolic_opset9.dropout(g, input, dropout_p, train)
        return symbolic_opset12.dropout(g, input, dropout_p, train)

In [3]:
def KL(input, target, reduction="sum"):
    input = input.float()
    target = target.float()
    loss = F.kl_div(F.log_softmax(input, dim=-1, dtype=torch.float32),
                    F.softmax(target, dtype=torch.float32), reduction=reduction)
    return loss

In [4]:
class FocalLoss(nn.Module):

    def __init__(self, weight=None, reduction='mean', gamma=0, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.eps = eps
        self.ce = torch.nn.CrossEntropyLoss(weight=weight, reduction=reduction)

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()

In [5]:
class DebertaV2ForMultipleChoice(DebertaV2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)

        num_labels = getattr(config, "num_labels", 2)
        self.num_labels = num_labels

        self.deberta = DebertaV2Model(config)
        self.pooler = ContextPooler(config)
        output_dim = self.pooler.output_dim

        self.classifier = nn.Linear(output_dim, 1)
        drop_out = getattr(config, "cls_dropout", None)
        drop_out = self.config.hidden_dropout_prob if drop_out is None else drop_out
        self.dropout = StableDropout(drop_out)

        self.init_weights()

    def get_input_embeddings(self):
        return self.deberta.get_input_embeddings()

    def set_input_embeddings(self, new_embeddings):
        self.deberta.set_input_embeddings(new_embeddings)

#     @add_start_docstrings_to_model_forward(DEBERTA_INPUTS_DOCSTRING.format("batch_size, sequence_length"))
#     @add_code_sample_docstrings(
#         checkpoint=_CHECKPOINT_FOR_DOC,
#         output_type=MultipleChoiceModelOutput,
#         config_class=_CONFIG_FOR_DOC,
#     )
    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, MultipleChoiceModelOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the multiple choice classification loss. Indices should be in `[0, ...,
            num_choices-1]` where `num_choices` is the size of the second dimension of the input tensors. (See
            `input_ids` above)
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        num_choices = input_ids.shape[1] if input_ids is not None else inputs_embeds.shape[1]

        flat_input_ids = input_ids.view(-1, input_ids.size(-1)) if input_ids is not None else None
        flat_position_ids = position_ids.view(-1, position_ids.size(-1)) if position_ids is not None else None
        flat_token_type_ids = token_type_ids.view(-1, token_type_ids.size(-1)) if token_type_ids is not None else None
        flat_attention_mask = attention_mask.view(-1, attention_mask.size(-1)) if attention_mask is not None else None
        flat_inputs_embeds = (
            inputs_embeds.view(-1, inputs_embeds.size(-2), inputs_embeds.size(-1))
            if inputs_embeds is not None
            else None
        )

        outputs = self.deberta(
            flat_input_ids,
            position_ids=flat_position_ids,
            token_type_ids=flat_token_type_ids,
            attention_mask=flat_attention_mask,
            inputs_embeds=flat_inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        encoder_layer = outputs[0]
        pooled_output = self.pooler(encoder_layer)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        reshaped_logits = logits.view(-1, num_choices)

        kl_outputs = self.deberta(
            flat_input_ids,
            position_ids=flat_position_ids,
            token_type_ids=flat_token_type_ids,
            attention_mask=flat_attention_mask,
            inputs_embeds=flat_inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        kl_output = kl_outputs[0]
        pooled_output = self.pooler(kl_output)
        pooled_output = self.dropout(pooled_output)
        kl_logits = self.classifier(pooled_output)

        reshaped_logits_kl = kl_logits.view(-1, num_choices)
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(reshaped_logits, labels)
            ce_loss = loss_fct(reshaped_logits_kl, labels)
            kl_loss = (KL(logits, kl_logits, "sum") + KL(kl_logits, logits, "sum")) / 2.
            focal_loss_fct = FocalLoss()
            focal_loss = focal_loss_fct(reshaped_logits, labels)
            total_loss = loss + ce_loss + kl_loss + focal_loss
        if not return_dict:
            output = (reshaped_logits,) + outputs[1:]
            return ((loss,) + output) if loss is not None else output

        return MultipleChoiceModelOutput(
            loss=loss,
            logits=reshaped_logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [6]:
import numpy as np
import pandas as pd
import datasets
from sklearn.model_selection import train_test_split

from transformers import Trainer, TrainingArguments
import os
import sys
import logging
from transformers import AutoTokenizer,DataCollatorWithPadding,DebertaV2Tokenizer
import torch

SP_train=np.load('/content/WP-train.npy',allow_pickle=True)
SP_test=np.load('/content/WP_new_test.npy',allow_pickle=True)

def deal(examples):
    key=examples[0].keys()
    question=[]
    choice_list=[]
    label=[]
    for i in range(len(examples)):
        if 'label' in key:
            label.append(int(examples[i]['label']))
        question.append(examples[i]['question'])
        choice_list.append(examples[i]['choice_list'])
    if 'label' in key:
        data_dict={'question':question,'choice_list':choice_list,'label':label}
    else:
        data_dict={'question':question,'choice_list':choice_list}
    return data_dict
def preprocess_function(examples):
    inputs=[]
    for i in range(len(examples['question'])):
        encodings=tokenizer([examples['question'][i]]*4, examples['choice_list'][i],
                            truncation=True,padding='max_length',max_length=96,return_tensors="pt")
        inputs.append(encodings)
    input_ids = torch.stack([x['input_ids'] for x in inputs])
    attention_mask = torch.stack([x['attention_mask'] for x in inputs])
    token_type_ids = torch.stack([x['token_type_ids'] for x in inputs])
    return {'input_ids':input_ids,'attention_mask':attention_mask,'token_type_ids':token_type_ids}
train_dict=deal(SP_train)
test_dict=deal(SP_test)
train, val = train_test_split(pd.DataFrame(train_dict), test_size=.2)
train=train.to_dict('list')
val=val.to_dict('list')
train_dataset = datasets.Dataset.from_dict(train)
val_dataset = datasets.Dataset.from_dict(val)
test_dataset=datasets.Dataset.from_dict(test_dict)

In [7]:
# import numpy as np
# import pandas as pd
# import datasets
# from sklearn.model_selection import train_test_split
# from transformers import AutoTokenizer, BertForMultipleChoice,DataCollatorWithPadding
# from transformers import Trainer, TrainingArguments
# import os
# import sys
# import logging
# import torch
# SP_train=pd.read_csv('/content/SP_train_translation.csv')
# SP_test=pd.read_csv('/content/SP_test_translation.csv')
# SP_train=SP_train.drop(columns='Unnamed: 0')
# SP_test=SP_test.drop(columns='Unnamed: 0')
# def create_choice_list(data):
#     choice_list=[]
#     for i in range(len(data['question'])):
#         choice_list.append([data['A'][i],data['B'][i],data['C'][i],data['D'][i]])
#     return choice_list
# SP_train['choice_list']=create_choice_list(SP_train)
# SP_test['choice_list']=create_choice_list(SP_test)
# SP_train=SP_train.drop(columns=['A','B','C','D'])
# SP_test=SP_test.drop(columns=['A','B','C','D'])

# train, val = train_test_split(pd.DataFrame(SP_train), test_size=.2)
# train=train.to_dict('list')
# val=val.to_dict('list')
# test_dict=SP_test.to_dict('list')
# train_dataset = datasets.Dataset.from_dict(train)
# val_dataset = datasets.Dataset.from_dict(val)
# test_dataset=datasets.Dataset.from_dict(test_dict)

# def preprocess_function(examples):
#     prompts=[]
#     inputs=[]
#     for i in range(len(examples['question'])):
#         encodings=tokenizer([examples['question'][i]]*4, examples['choice_list'][i],
#                             truncation=True,padding='max_length',max_length=96,return_tensors="pt")
#         inputs.append(encodings)
#     input_ids = torch.stack([x['input_ids'] for x in inputs])
#     attention_mask = torch.stack([x['attention_mask'] for x in inputs])
#     token_type_ids = torch.stack([x['token_type_ids'] for x in inputs])
#     return {'input_ids':input_ids,'attention_mask':attention_mask,'token_type_ids':token_type_ids}

In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels.flatten(), preds.flatten(), average='weighted', zero_division=0)
    return {
        'accuracy': (preds == eval_pred.label_ids).mean(),
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

In [9]:
from sklearn.metrics import precision_recall_fscore_support
import warnings
from transformers import AutoTokenizer
import evaluate
warnings.filterwarnings('ignore')
check_point="microsoft/deberta-v2-xxlarge"
tokenizer = AutoTokenizer.from_pretrained(check_point)
model = DebertaV2ForMultipleChoice.from_pretrained(check_point)

tokenized_train = preprocess_function(train_dataset)
tokenized_val = preprocess_function(val_dataset)
tokenized_test = preprocess_function(test_dataset)
labels01=torch.Tensor(train_dataset['label']).cuda()
labels02=torch.Tensor(val_dataset['label']).cuda()
tokenized_train['label']=labels01.int()
tokenized_val['label']=labels02.int()

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

metric = evaluate.load("accuracy")

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/633 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.45M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.14G [00:00<?, ?B/s]

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v2-xxlarge and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.weight', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
training_args = TrainingArguments(
        output_dir='./checkpoint',  # output directory
        num_train_epochs=5,  # total number of training epochs
        per_device_train_batch_size=1,  # batch size per device during training
        per_device_eval_batch_size=1,  # batch size for evaluation
        warmup_steps=500,  # number of warmup steps for learning rate scheduler
        weight_decay=0.01,  # strength of weight decay
        logging_dir='./logs',  # directory for storing logs
        logging_steps=100,
        save_strategy='epoch',
        evaluation_strategy="epoch",
        learning_rate=5e-5,
        fp16=True,
        remove_unused_columns=False,
        #load_best_model_at_end=True

)

trainer = Trainer(
        model=model,  # the instantiated 🤗 Transformers model to be trained
        args=training_args,  # training arguments, defined above
        train_dataset=datasets.Dataset.from_dict(tokenized_train),  # training dataset
        eval_dataset=datasets.Dataset.from_dict(tokenized_val),  # evaluation dataset
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

In [11]:
import torch.nn.functional as F
trainer.train()
print(trainer.evaluate())


You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.365000,1.386218,0.400000,0.379400,0.613509,0.400000
2,1.443300,1.386182,0.350000,0.305802,0.467011,0.350000
3,1.399800,1.386414,0.300000,0.309913,0.366647,0.300000
4,1.387400,1.386475,0.237500,0.268048,0.308122,0.237500
5,1.428700,1.386707,0.287500,0.144529,0.277564,0.287500


{'eval_loss': 1.3867065906524658, 'eval_accuracy': 0.2875, 'eval_f1': 0.1445294117647059, 'eval_precision': 0.2775641025641026, 'eval_recall': 0.2875, 'eval_runtime': 15.9523, 'eval_samples_per_second': 5.015, 'eval_steps_per_second': 5.015, 'epoch': 5.0}


In [12]:
prediction_outputs = trainer.predict(datasets.Dataset.from_dict(tokenized_test))
test_pred = np.argmax(prediction_outputs[0], axis=-1).flatten()
print(test_pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
